### Praproses Data

1. Penyesuaian nama kolom dan bulan dengan bahasa Inggris
2. Penambahan atribut tahun dan kota

#### Data cuaca dan curah hujan kota Bandung
##### Hadoop

In [2]:
!ls

lingkungan_hidup_bandung  Lingkungan Hidup Bandung.ipynb


In [14]:
!hdfs dfs -ls /user/hadoop/lh_bandung/raw_data/

18/12/18 09:53:15 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Found 18 items
-rw-r--r--   1 hadoop supergroup        485 2018-12-18 09:53 /user/hadoop/lh_bandung/raw_data/cuaca-dan-curah-hujan-di-kota-bandung-menurut-bulan-tahun-2013.csv
-rw-r--r--   1 hadoop supergroup        518 2018-12-18 09:53 /user/hadoop/lh_bandung/raw_data/cuaca-dan-curah-hujan-di-kota-bandung-menurut-bulan-tahun-2014.csv
-rw-r--r--   1 hadoop supergroup        620 2018-12-18 09:53 /user/hadoop/lh_bandung/raw_data/cuaca-dan-curah-hujan-di-kota-bandung-menurut-bulan-tahun-2015.csv
-rw-r--r--   1 hadoop supergroup        670 2018-12-18 09:53 /user/hadoop/lh_bandung/raw_data/cuaca-dan-curah-hujan-di-kota-bandung-menurut-bulan-tahun-2016.csv
-rw-r--r--   1 hadoop supergroup        595 2018-12-18 09:53 /user/hadoop/lh_bandung/raw_data/cuaca-dan-curah-hujan-di-kota-bandung-menurut-bulan-tahun-2017.csv
-rw-r--r--   1 hadoop supergroup

In [13]:
!hdfs dfs -put lingkungan_hidup_bandung/raw_data/* /user/hadoop/lh_bandung/raw_data/

18/12/18 09:53:08 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
# hapus file yang ada didalam direktori
!hdfs dfs -rmr /user/hadoop/lh_bandung/clean_data/*

### Spark dengan Pyspark

In [27]:
# spark pakai pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.functions import format_number

In [2]:
# buat spark session
spark = SparkSession.builder.appName('Basics').getOrCreate()

In [57]:
# read file csv di lokal
df_13 = spark.read.json('lingkungan_hidup_bandung/clean_data/json/rainfall_bandung_2013.json')
df_14 = spark.read.json('lingkungan_hidup_bandung/clean_data/json/rainfall_bandung_2014.json')
df_15 = spark.read.json('lingkungan_hidup_bandung/clean_data/json/rainfall_bandung_2015.json')
df_16 = spark.read.json('lingkungan_hidup_bandung/clean_data/json/rainfall_bandung_2016.json')
df_17 = spark.read.json('lingkungan_hidup_bandung/clean_data/json/rainfall_bandung_2017.json')

In [3]:
df = spark.read.csv('air_condition_bandung.csv', header='true', inferSchema='true')

In [ ]:
# read file csv dari hdfs
df = sqlContext.read.format('com.databricks.spark.csv').\
options(header='true', inferSchema='true').\
load("hdfs://localhost:9000/user/hadoop/lh_bandung/raw_data/cuaca-dan-curah-hujan-di-kota-bandung-menurut-bulan-tahun-2016.csv")

In [ ]:
# read file csv dari hdfs
df = spark.read.format("csv")\
.option("delimiter", ",")\
.option("header", True)\
.load("hdfs://localhost:9000/user/hadoop/lh_bandung/raw_data/cuaca-dan-curah-hujan-di-kota-bandung-menurut-bulan-tahun-2017.csv")

In [38]:
# import modul tambahan
import json
import seaborn as sb
import pandas as pd
from matplotlib import pyplot as plt

### Proses Data Cleaning

In [58]:
# tampilkan dataframe
df.show()

+------------+-------+-----------+---------+-----------------+----+
|air_pressure|   city|evaporation|   months|relative_humidity|year|
+------------+-------+-----------+---------+-----------------+----+
|       923.3|Bandung|        3.9|  January|               79|2013|
|       922.1|Bandung|        3.7| February|               80|2013|
|       923.4|Bandung|        4.2|    March|               79|2013|
|       922.7|Bandung|        3.2|    April|               82|2013|
|       922.9|Bandung|        3.3|      May|               82|2013|
|       922.1|Bandung|        3.4|     June|               78|2013|
|       923.2|Bandung|        3.1|     July|               77|2013|
|       923.9|Bandung|        4.2|   August|               71|2013|
|       923.8|Bandung|        4.4|September|               70|2013|
|       924.5|Bandung|        4.4|  October|               73|2013|
|       922.5|Bandung|        4.0| November|               74|2013|
|       922.5|Bandung|        3.5| December|    

In [62]:
# union dataframe
join = join.union(df_17)

In [63]:
join.show()

+---+--------+-------+--------+--------+---------+--------+---------+----+
|LPM|avg_temp|   city|max_temp|min_temp|   months|rainfall|rainy_day|year|
+---+--------+-------+--------+--------+---------+--------+---------+----+
| 49|    23.4|Bandung|    28.3|    21.0|  January|   216.9|       26|2013|
| 52|    23.4|Bandung|    28.6|    20.3| February|   250.0|       23|2013|
| 58|    23.8|Bandung|    29.6|    20.4|    March|   305.0|       24|2013|
| 52|     3.7|Bandung|    29.1|    20.7|    April|   286.0|       26|2013|
| 53|    23.5|Bandung|    28.7|    20.3|      May|   171.0|       23|2013|
| 61|    23.6|Bandung|    28.5|    20.3|     June|   231.5|       16|2013|
| 58|    22.5|Bandung|    28.0|    19.0|     July|   159.0|       16|2013|
| 79|     3.2|Bandung|    29.4|    18.5|   August|    74.0|        9|2013|
| 77|    23.7|Bandung|    30.1|    19.1|September|   172.0|       10|2013|
| 69|    23.8|Bandung|    30.0|    19.6|  October|   234.0|       21|2013|
| 64|    23.8|Bandung|   

In [11]:
group = df.groupBy('year')

In [13]:
group.avg().show()

+----+-----------------+------------------+----------------------+---------+
|year|avg(air_pressure)|  avg(evaporation)|avg(relative_humidity)|avg(year)|
+----+-----------------+------------------+----------------------+---------+
|2015|924.1166666666667| 3.891666666666666|                  74.5|   2015.0|
|2013|923.0749999999999|             3.775|     77.08333333333333|   2013.0|
|2014|923.7416666666668| 3.608333333333333|     77.16666666666667|   2014.0|
|2016|923.4500000000002| 3.216666666666667|     79.83333333333333|   2016.0|
|2017|923.0666666666666|3.3249999999999997|     77.33333333333333|   2017.0|
+----+-----------------+------------------+----------------------+---------+



In [64]:
yearavg = join.select(['LPM','avg_temp', 'max_temp','min_temp','rainfall','rainy_day','year']).groupBy('year').avg()

In [65]:
yearavg = yearavg.drop('avg(year)')

In [66]:
yearavg.show()

+----+------------------+------------------+------------------+------------------+------------------+------------------+
|year|          avg(LPM)|     avg(avg_temp)|     avg(max_temp)|     avg(min_temp)|     avg(rainfall)|    avg(rainy_day)|
+----+------------------+------------------+------------------+------------------+------------------+------------------+
|2014|60.333333333333336|23.391666666666666|29.133333333333336| 19.76666666666667|198.77499999999998|18.833333333333332|
|2016|51.583333333333336|23.749999999999996|29.224999999999998|             20.55| 295.7583333333333|              25.0|
|2017|             56.25|23.516666666666666|29.183333333333334|            20.125|191.54166666666666|             19.75|
|2013|59.333333333333336|            20.125|29.041666666666668|19.941666666666666|223.45000000000002|              20.0|
|2015|              67.0| 23.53333333333333|29.849999999999998|19.491666666666664| 184.7416666666667|17.416666666666668|
+----+------------------+-------

In [67]:
yearavg = yearavg.withColumnRenamed('avg(LPM)','LPM')
yearavg = yearavg.withColumnRenamed('avg(avg_temp)','avg_temp')
yearavg = yearavg.withColumnRenamed('avg(max_temp)','max_temp')
yearavg = yearavg.withColumnRenamed('avg(min_temp)','min_temp')
yearavg = yearavg.withColumnRenamed('avg(rainfall)','rainfall')
yearavg = yearavg.withColumnRenamed('avg(rainy_day)','rainy_day')

In [68]:
yearavg = yearavg.select('year',format_number('LPM', 2).alias('LPM'),
                         format_number('avg_temp', 2).alias('avg_temp'),
                         format_number('max_temp', 2).alias('max_temp'),
                         format_number('min_temp', 2).alias('min_temp'),
                         format_number('rainfall', 2).alias('rainfall'),
                         format_number('rainy_day', 2).alias('rainy_day')).orderBy('year')

In [69]:
yearavg.printSchema()

root
 |-- year: long (nullable = true)
 |-- LPM: string (nullable = true)
 |-- avg_temp: string (nullable = true)
 |-- max_temp: string (nullable = true)
 |-- min_temp: string (nullable = true)
 |-- rainfall: string (nullable = true)
 |-- rainy_day: string (nullable = true)



In [8]:
# print skema dataframe
df.printSchema()

root
 |-- air_pressure: double (nullable = true)
 |-- city: string (nullable = true)
 |-- evaporation: double (nullable = true)
 |-- months: string (nullable = true)
 |-- relative_humidity: long (nullable = true)
 |-- year: long (nullable = true)



In [ ]:
# select data tertentu dari dataframe
df = df.select('months', 'avg_temp', 'max_temp', 'min_temp', 'rainfall', 'rainy_day', 'LPM', 'city', 'year')

In [ ]:
# untuk menghapus missing value
df = df.dropna()

In [ ]:
# tambahkan kolom data baru
df = df.withColumn('city',lit('Bandung'))
df = df.withColumn('year',lit('2017'))

In [ ]:
# ganti nama kolom di dataframe
df = df.withColumnRenamed('Bulan', 'months')
df = df.withColumnRenamed('Temperatur Rata-rata (oC)', 'avg_temp')
df = df.withColumnRenamed('Temperatur Maksimal (oC)', 'max_temp')
df = df.withColumnRenamed('Temperatur Minimal (oC)', 'min_temp')
df = df.withColumnRenamed('Jumlah Curah Hujan (mm)', 'rainfall')
df = df.withColumnRenamed('Hari Hujan (hari)', 'rainy_day')
df = df.withColumnRenamed('LPM/Sunshine Duration (%)', 'LPM')

##### export data jadi file json

In [70]:
# export dataframe ke json
results = yearavg.toJSON().map(lambda j: json.loads(j)).collect()

In [71]:
# write file json ke lokal
with open('rainfall_bandung.json', 'w') as f:
     json.dump(results, f)

In [ ]:
# save file csv ke hdfs
df.coalesce(1).write.csv('hdfs://localhost:9000/user/hadoop/lh_bandung/clean_data/rainfall_bandung_2014.csv', header='true')

In [72]:
# save file csv ke local
join.coalesce(1).write.format('csv').save('rainfall_bandung.csv',header = 'true')

#### Data keadaan udara kota Bandung

In [254]:
#read data yang ada di hdfs
df = spark.read.format("csv")\
.option("delimiter", ",")\
.option("header", True)\
.load("hdfs://localhost:9000/user/hadoop/lh_bandung/raw_data/keadaan-udara-menurut-bulan-di-kota-bandung-2017.csv")

In [45]:
# read file csv di lokal
df = spark.read.csv('lingkungan_hidup_bandung/clean_data/air_condition_bandung_2017.csv', header='true', inferSchema='true')

In [255]:
df.show()

+-----+---------+--------------+------------------+--------------------+----+----+----+----+----+----+----+
|Nomor|B u l a n|Penguapan (mm)|Tekanan Udara (mb)|Kelembapan Nisbi (%)| _c5| _c6| _c7| _c8| _c9|_c10|_c11|
+-----+---------+--------------+------------------+--------------------+----+----+----+----+----+----+----+
|    1|  Januari|           3.6|             922.3|                  75|null|null|null|null|null|null|null|
|    2| Februari|             3|             923.2|                  81|null|null|null|null|null|null|null|
|    3|    Maret|           3.5|             923.3|                  80|null|null|null|null|null|null|null|
|    4|    April|             3|             923.4|                  81|null|null|null|null|null|null|null|
|    5|      Mei|           3.1|             923.3|                  78|null|null|null|null|null|null|null|
|    6|     Juni|           2.6|             923.9|                  79|null|null|null|null|null|null|null|
|    7|     Juli|           

In [256]:
# kolom bahasa inggris
# months, evaporation, air pressure, relative humidity
#tambahkan kolom data baru
df = df.withColumn('city',lit('Bandung'))
df = df.withColumn('year',lit('2017'))

In [260]:
#ganti nama kolom
df = df.withColumnRenamed('B u l a n', 'months')
df = df.withColumnRenamed('Penguapan (mm)', 'evaporation')
df = df.withColumnRenamed('Tekanan udara (mb)', 'air_pressure')
df = df.withColumnRenamed('Kelembapan Nisbi (%)', 'relative_humidity')

In [262]:
df = df.select('months','evaporation','air_pressure','relative_humidity','city','year')

In [263]:
#save ke local
df.coalesce(1).write.format('csv').save('air_condition_bandung_2017.csv',header = 'true')

In [46]:
# export dataframe ke json
results = df.toJSON().map(lambda j: json.loads(j)).collect()

In [47]:
# write file json ke lokal
with open('air_condition_bandung_2017.json', 'w') as f:
     json.dump(results, f)

In [54]:
df_json = df.toJSON()
row = df_json.collect()

In [131]:
with open('JSONData.json', 'w') as f:
     json.dump(results, f)

In [61]:
# toJSON() turns each row of the DataFrame into a JSON string
# calling first() on the result will fetch the first row.
results = json.loads(df.toJSON().first())

#for key in results:
#    print (results['rainfall'])
results

{'LPM': 49,
 'avg_temp': 23.4,
 'city': 'Bandung',
 'max_temp': 28.3,
 'min_temp': 21.0,
 'months': 'January',
 'rainfall': 216.9,
 'rainy_day': 26,
 'year': 2013}

In [16]:
# To decode the entire DataFrame iterate over the result
# of toJSON()

def print_rows(row):
    data = json.loads(row)
    for key in data:
        print ("{key}:{value}".format(key=key, value=data[key]))

In [76]:
data = json.loads(json.dumps(row))

#### Data kecepatan angin Bandung

In [95]:
# read file csv di lokal
df = spark.read.csv('lingkungan_hidup_bandung/raw_data/kecepatan-angin-menurut-bulan-di-kota-bandung-2017.csv', header='true', inferSchema='true')

In [96]:
df.show()

+---------+--------------------------------+-------------------------------+
|    Bulan|Kecepatan Angin Rata-rata (knot)|Kecepatan Angin Terbesar (knot)|
+---------+--------------------------------+-------------------------------+
|  January|                               5|                             19|
| February|                               5|                             16|
|    March|                               5|                             12|
|    April|                               4|                             15|
|      May|                               4|                             10|
|     June|                               4|                             10|
|     July|                               5|                             10|
|   August|                               5|                              8|
|September|                               4|                              8|
|  October|                               4|                             13|

In [97]:
# kolom bahasa inggris
# months, evaporation, air pressure, relative humidity
#tambahkan kolom data baru
df = df.withColumn('city',lit('Bandung'))
df = df.withColumn('year',lit('2017'))

In [98]:
df.show()

+---------+--------------------------------+-------------------------------+-------+----+
|    Bulan|Kecepatan Angin Rata-rata (knot)|Kecepatan Angin Terbesar (knot)|   city|year|
+---------+--------------------------------+-------------------------------+-------+----+
|  January|                               5|                             19|Bandung|2017|
| February|                               5|                             16|Bandung|2017|
|    March|                               5|                             12|Bandung|2017|
|    April|                               4|                             15|Bandung|2017|
|      May|                               4|                             10|Bandung|2017|
|     June|                               4|                             10|Bandung|2017|
|     July|                               5|                             10|Bandung|2017|
|   August|                               5|                              8|Bandung|2017|
|September

In [99]:
#ganti nama kolom
df = df.withColumnRenamed('Bulan', 'months')
df = df.withColumnRenamed('Kecepatan Angin Rata-rata (knot)', 'avg_speed')
df = df.withColumnRenamed('Kecepatan Angin Terbesar (knot)', 'max_speed')

In [100]:
df.show()

+---------+---------+---------+-------+----+
|   months|avg_speed|max_speed|   city|year|
+---------+---------+---------+-------+----+
|  January|        5|       19|Bandung|2017|
| February|        5|       16|Bandung|2017|
|    March|        5|       12|Bandung|2017|
|    April|        4|       15|Bandung|2017|
|      May|        4|       10|Bandung|2017|
|     June|        4|       10|Bandung|2017|
|     July|        5|       10|Bandung|2017|
|   August|        5|        8|Bandung|2017|
|September|        4|        8|Bandung|2017|
|  October|        4|       13|Bandung|2017|
| November|        5|       16|Bandung|2017|
| December|        6|       16|Bandung|2017|
+---------+---------+---------+-------+----+



In [101]:
# export dataframe ke json
results = df.toJSON().map(lambda j: json.loads(j)).collect()

In [102]:
# write file json ke lokal
with open('wind_bandung_2017.json', 'w') as f:
     json.dump(results, f)